In [3]:
import pandas as pd
import dash
import numpy as np
import plotly.express as px
import pandas_profiling

# Import the csv file

In [9]:
import pandas_profiling

In [248]:
df=pd.read_csv('BPD_Part_1_Victim_Based_Crime_Data.csv')

D:\Python37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning:

Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.



# Data Integrity Check

In [10]:
# generating the EDA 
df.profile_report(title='Pandas Profiling Report').to_file(output_file="eda.html")

* Duplicates of records
* High correlation between some variables
* Missing values on two variables

## Grouping the records to keep the number of incidents but drop duplicates

In [249]:
df_nodup=df.fillna('')\
      .groupby(df.columns.tolist()).apply(len)\
      .rename('group_count')\
      .reset_index()\
      .replace('',np.nan)\
      .sort_values(by = ['group_count'], ascending = False)

## ~~ Convert the Data/Time to proper time format~~

In [250]:
# df_nodup.CrimeDate=pd.to_datetime(df_nodup.CrimeDate)
# df_nodup.CrimeTime=pd.to_timedelta(df_nodup.CrimeTime)
# df_nodup.CrimeTime_dt=pd.to_datetime(df_nodup.CrimeTime)

## Handling missing values

In [251]:
df_nodup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296586 entries, 215100 to 296585
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   CrimeDate        296586 non-null  object 
 1   CrimeTime        296561 non-null  object 
 2   CrimeCode        296586 non-null  object 
 3   Location         295207 non-null  object 
 4   Description      296586 non-null  object 
 5   Inside/Outside   260776 non-null  object 
 6   Weapon           59666 non-null   object 
 7   Post             294212 non-null  object 
 8   District         296586 non-null  object 
 9   Neighborhood     286259 non-null  object 
 10  Longitude        294900 non-null  float64
 11  Latitude         294900 non-null  float64
 12  Location 1       0 non-null       float64
 13  Premise          260538 non-null  object 
 14  vri_name1        35204 non-null   object 
 15  Total Incidents  296586 non-null  int64  
 16  group_count      296586 non-null 

In [252]:
df_nodup[df_nodup['vri_name1'].notna()].head(10)

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,vri_name1,Total Incidents,group_count
2922,1/12/2015,4:40:00,4A,1700 NORMAL AV,AGG. ASSAULT,I,FIREARM,312,EASTERN,DARLEY PARK,-76.593663,39.315562,NaN,ROW/TOWNHOUSE-OCC,Eastern 1,1,10
123765,3/17/2020,18:27:00,4A,1800 RAMSAY ST,AGG. ASSAULT,NaN,FIREARM,934,SOUTHERN,CARROLLTON RIDGE,-76.645249,39.283106,NaN,NaN,Tri-District,1,9
271597,8/9/2017,20:22:00,4C,2100 HOLLINS ST,AGG. ASSAULT,I,OTHER,835,SOUTHWEST,BOOTH-BOYD,-76.650689,39.287050,NaN,ROW/TOWNHOUSE-OCC,Tri-District,1,9
89916,12/3/2014,15:11:00,4C,1000 EDMONDSON AV,AGG. ASSAULT,I,OTHER,713,WESTERN,HARLEM PARK,-76.634581,39.295513,NaN,ROW/TOWNHOUSE-OCC,Central,1,8
179796,5/25/2015,0:01:00,6E,5100 REISTERSTOWN RD,LARCENY,O,NaN,613,NORTHWEST,LANGSTON HUGHES,-76.682544,39.342345,NaN,PARKING LOT-OUTSIDE,Northwestern,1,8
228996,7/19/2017,16:33:00,3JF,1600 N BROADWAY,ROBBERY - RESIDENCE,I,FIREARM,314,EASTERN,OLIVER,-76.595103,39.308805,NaN,ROW/TOWNHOUSE-OCC,Eastern 1,1,6
155708,4/27/2015,16:00:00,4C,1600 W NORTH AVE,AGG. ASSAULT,O,OTHER,733,WESTERN,PENN NORTH,-76.643429,39.310091,NaN,STREET,Western,1,6
184451,5/3/2016,3:33:00,4A,2200 E BIDDLE ST,AGG. ASSAULT,O,FIREARM,331,EASTERN,BROADWAY EAST,-76.586960,39.304670,NaN,STREET,Eastern 2,1,6
93188,12/5/2018,20:44:00,4E,2400 W BALTIMORE ST,COMMON ASSAULT,I,NaN,714,WESTERN,SHIPLEY HILL,-76.655035,39.287728,NaN,HOSP/NURS. HOME,Tri-District,1,6
29289,10/14/2015,0:02:00,4C,1000 DRUID HILL AVE,AGG. ASSAULT,I,OTHER,123,CENTRAL,UPTON,-76.626366,39.300154,NaN,ROW/TOWNHOUSE-OCC,Central,1,6


In [253]:
# df_nodup[df_nodup['vri_name1'].notna()][['vri_name1','District']].
pd.pivot_table(index=['vri_name1'], columns=['District'], data=df_nodup[df_nodup['vri_name1'].notna()][['vri_name1','District','Total Incidents']], aggfunc=np.sum)

Total Incidents                                                 \
District             CENTRAL EASTERN NORTHEAST NORTHERN NORTHWEST SOUTHEAST   
vri_name1                                                                     
Brooklyn                 NaN     NaN       NaN      NaN       1.0       NaN   
Central               2783.0    29.0       NaN      1.0       1.0       NaN   
Eastern 1               11.0  4611.0     184.0      NaN       NaN       NaN   
Eastern 2                4.0  3398.0       NaN      1.0       NaN      31.0   
Northeastern             NaN     1.0    2981.0      NaN       NaN       NaN   
Northwestern             1.0     NaN       NaN      1.0    3254.0       NaN   
Tri-District             NaN     NaN       NaN      NaN       1.0      73.0   
Western                  8.0    34.0       2.0      NaN       2.0       NaN   

                                                 
District     SOUTHERN SOUTHWEST UNKNOWN WESTERN  
vri_name1                                        
Brooklyn       2471.0       1.0     2.0     NaN  
Central           5.0       1.0     1.0  2677.0  
Eastern 1         NaN       1.0     2.0    19.0  
Eastern 2         NaN       NaN     1.0     NaN  
Northeastern      NaN       NaN     1.0     NaN  
Northwestern      NaN       NaN     NaN     NaN  
Tri-District   3121.0    2624.0     2.0   927.0  
Western           7.0       1.0     2.0  5925.0

In [254]:
df_nodup=df_nodup.drop(columns=['Location 1','vri_name1'])

## Value Consistency

In [255]:
df_nodup.CrimeCode.unique()

array(['5D', '5E', '4C', '6D', '5A', '4A', '4E', '3AF', '6E', '6J', '9S',
       '3JF', '3CF', '3NF', '6A', '6G', '3B', '4B', '5F', '8AO', '3P',
       '3AO', '6C', '8AV', '3AK', '7A', '5G', '3JO', '4D', '5C', '7C',
       '2A', '6L', '1A', '3GK', '8GO', '6F', '3BJ', '8H', '3AJF', '3K',
       '3AJK', '3GF', '8BO', '3JK', '3F', '1K', '5B', '3EF', '3D', '3NK',
       '3LF', '3CO', '6B', '8J', '3CK', '8FO', '1O', '8EO', '8BV', '6H',
       '3NO', '3GO', '3AJO', '3M', '7B', '3EO', '3H', '2B', '3EK', '3LO',
       '8CO', '6K', '8GV', '8CV', '3LK', '8FV', '8EV', '8I', '8DO', '3N'],
      dtype=object)

In [256]:
df_nodup.Description.unique()

array(['BURGLARY', 'AGG. ASSAULT', 'LARCENY FROM AUTO', 'COMMON ASSAULT',
       'ROBBERY - STREET', 'LARCENY', 'SHOOTING', 'ROBBERY - RESIDENCE',
       'ROBBERY - COMMERCIAL', 'ARSON', 'AUTO THEFT', 'RAPE', 'HOMICIDE',
       'ROBBERY - CARJACKING'], dtype=object)

In [257]:
#Before Mapping

In [258]:
df_nodup['Inside/Outside'].unique()

array(['I', nan, 'O', 'Outside', 'Inside'], dtype=object)

In [259]:
df_nodup['Inside/Outside']=df_nodup['Inside/Outside'].map({'O':'Outside','I':'Inside'})

In [260]:
#After Mapping

In [261]:
df_nodup['Inside/Outside'].unique()

array(['Inside', nan, 'Outside'], dtype=object)

In [262]:
# cleaning posts

In [263]:
df_nodup.loc[df_nodup['Post'].notna(),'Post']=df_nodup[df_nodup['Post'].notna()]['Post'].apply(lambda x: str(x).replace('.0',''))

In [264]:
df_nodup['Post'].unique()

array(['922', '232', '533', '731', '832', '835', '912', '625', '632',
       '426', '724', '813', '212', '441', '411', '423', '445', '714',
       '941', '511', '312', '622', '634', '614', nan, '132', '934', '125',
       '833', '334', '134', '713', '313', '633', '923', '225', '322',
       '613', '231', '711', '624', '222', '523', '413', '721', '443',
       '323', '942', '114', '425', '124', '113', '723', '822', '433',
       '424', '121', '621', '211', '712', '422', '314', '512', '824',
       '221', '534', '733', '331', '233', '412', '123', '631', '812',
       '214', '324', '514', '821', '931', '521', '911', '111', '722',
       '224', '414', '234', '513', '213', '442', '532', '834', '321',
       '943', '525', '223', '432', '612', '831', '735', '734', '131',
       '112', '431', '935', '932', '814', '421', '623', '333', '921',
       '823', '913', '434', '311', '332', '524', '522', '515', '531',
       '444', '133', '611', '811', '815', '732', '415', '122', '933',
       'ED4', '

## Normalizing the Neigh.

* SANDTO ---> Sandtown
* HARFORD-ECHODALE-PER --->?
* CLAREMONT-FREEDOM -->?
* GLEN -->
* HOLLIN'S PARK -->
* JOSEPH LEE --> ?

In [27]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from shapely.geometry import shape, Point

In [72]:
df_nodup=pd.read_csv("cleaned_df.csv")

with open("Neighborhoods.geojson") as f:
    geojson = json.load(f)

D:\Python37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [73]:
def find_nei(pp):
    name=None
    for feature in geojson['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(pp):
            return feature['properties']['Name']
        else:
            continue
    return name
        

In [6]:
neib_ls=[]
for nei in geojson['features']:
    neib_ls.append(nei['properties']['Name'])

In [40]:
neigh_name_map={}
second_match_ls=[]
for nei_name in df_nodup[df_nodup.Neighborhood.notna()].Neighborhood.unique():
    print('Matching {}-----'.format(nei_name))
    re=process.extract(nei_name,neib_ls)
#     print('Matching {}-----'.format(nei_name))
#     print(re)
    if re[0][1]>90:
        neigh_name_map[nei_name]=re[0][0]
    else:
        print('Failed. Adding to the List')
        second_match_ls.append(nei_name)

Matching CHERRY HILL-----
Matching GRACELAND PARK-----
Matching WOODBERRY-----
Matching MONDAWMIN-----
Matching SAINT AGNES-----
Matching CARROLL-SOUTH HILTON-----
Matching SABINA-MATTFELDT-----
Matching ASHBURTON-----
Matching CROSS COUNTRY-----
Matching GWYNNS FALLS-----
Matching WALTHERSON-----
Matching SANDTOWN-WINCHESTER-----
Matching ROSEMONT-----
Matching DUNBAR-----
Failed. Adding to the List
Matching PULASKI INDUSTRIAL AREA-----
Matching LIBERTY SQUARE-----
Matching EAST BALTIMORE MIDWA-----
Matching HARFORD-ECHODALE-PER-----
Failed. Adding to the List
Matching COLDSTREAM HOMESTEAD-----
Failed. Adding to the List
Matching CEDMONT-----
Matching PENROSE/FAYETTE ST OUTREACH-----
Matching OTTERBEIN-----
Matching REMINGTON-----
Matching DARLEY PARK-----
Matching HOWARD PARK-----
Matching GROVE PARK-----
Matching CENTRAL PARK HEIGHTS-----
Matching BOLTON HILL-----
Matching CARROLLTON RIDGE-----
Matching BOOTH-BOYD-----
Matching SETON HILL-----
Matching IRVINGTON-----
Matching YALE H

Matching WOODME-----
Failed. Adding to the List
Matching VILLAGES OF HOMELAND-----
Matching BELAIR-----
Failed. Adding to the List
Matching CARROL-----
Failed. Adding to the List
Matching SANDTO-----
Failed. Adding to the List
Matching DUNDALK MARINE TERMINAL-----
Matching HOLLIN-----
Failed. Adding to the List
Matching 116102-----
Failed. Adding to the List


In [74]:
#Able to match 246 neigh names
len(neigh_name_map.values())

246

In [81]:
second_match_ls

['DUNBAR',
 'HARFORD-ECHODALE-PER',
 'COLDSTREAM HOMESTEAD',
 'HUDSON-HIGHLANDTOWN',
 'BALTIMORE-LINWOOD',
 'GLENHAM-BELFORD',
 'CLAREMONT-FREEDOM',
 'HOLLINS PARK',
 'DORCHESTER/GRAN-BARR',
 'JOSEPH LEE',
 'FEDERAL HILL/MONTGOM',
 'COPPIN HEIGHTS/ASHBU',
 'S.B.I.C.',
 '4X4',
 'MEDFORD/BROENING MAN',
 'WOODBROOK',
 'WOODRING',
 'CONCERNED CITIZENS O',
 'FAIRFIELD',
 'BRADDISH AVE./PANWAY',
 'ROSEMONT AVENUE',
 'WILSON HEIGHTS',
 'FORT HOLABIRD',
 'CHRISTOPHER',
 'STONEWOOD-PENTWOOD-W',
 '0',
 'WOODME',
 'BELAIR',
 'CARROL',
 'SANDTO',
 'HOLLIN',
 '116102']

In [75]:
df_nodup['Neighborhood_New']=df_nodup.Neighborhood.map(neigh_name_map)

In [76]:
len(df_nodup[df_nodup['Neighborhood_New'].isna()])

38486

In [69]:
t=df_nodup[df_nodup['Neighborhood_New'].isna()].apply(lambda x: find_nei(Point(x.Longitude, x.Latitude)), axis=1)

In [70]:
from collections import Counter
Counter(t).most_common()


[('Patterson Park Neighborhood', 3486),
 ('Hamilton Hills', 3353),
 ('Coldstream Homestead Montebello', 3266),
 ('Highlandtown', 2063),
 ('Glenham-Belhar', 1931),
 ('Dunbar-Broadway', 1638),
 ('Federal Hill', 1396),
 ('Hollins Market', 1316),
 ('Coppin Heights/Ash-Co-East', 1150),
 ('South Baltimore', 927),
 ('Parkview/Woodbrook', 862),
 ('Bayview', 843),
 ('Dorchester', 807),
 ('Broening Manor', 786),
 (None, 675),
 ('North Harford Road', 584),
 ('Rosemont East', 579),
 ('Four By Four', 572),
 ('Orchard Ridge', 504),
 ('Panway/Braddish Avenue', 443),
 ('Concerned Citizens Of Forest Park', 405),
 ('Fairfield Area', 369),
 ('Downtown', 366),
 ('Pulaski Industrial Area', 294),
 ('Waltherson', 261),
 ('Brooklyn', 254),
 ('Canton', 250),
 ('Stonewood-Pentwood-Winston', 241),
 ('McElderry Park', 172),
 ('Sandtown-Winchester', 165),
 ('Central Park Heights', 161),
 ('Frankford', 155),
 ('Westgate', 150),
 ('Fells Point', 149),
 ('Medford', 144),
 ('Ellwood Park/Monument', 140),
 ('Inner Harb

In [78]:
df_nodup.loc[df_nodup['Neighborhood_New'].isna(),'Neighborhood_New']=df_nodup[df_nodup['Neighborhood_New'].isna()].apply(lambda x: find_nei(Point(x.Longitude, x.Latitude)), axis=1)

## Normalize District

In [84]:
with open("Police_Districts.geojson") as f1:
    dis_geojson = json.load(f1)

In [91]:
dis_ls=[]
for dis in dis_geojson['features']:
    dis_ls.append(dis['properties']['dist_name'])

In [92]:
dis_ls

['Northern',
 'Central',
 'Western',
 'Eastern',
 'Southeastern',
 'Southern',
 'Southwestern',
 'Northwestern',
 'Northeastern']

In [94]:
df_nodup.District.unique()

array(['SOUTHERN', 'SOUTHEAST', 'NORTHERN', 'WESTERN', 'SOUTHWEST',
       'NORTHWEST', 'NORTHEAST', 'EASTERN', 'CENTRAL', 'UNKNOWN'],
      dtype=object)

In [100]:
dist_name_map={}
# dist_second_match_ls=[]
for dist in df_nodup[df_nodup.District.notna()].District.unique():
    print('Matching {}-----'.format(dist))
    re=process.extract(dist,dis_ls)
#     print('Matching {}-----'.format(nei_name))
#     print(re)
    if re[0][1]>80:
        dist_name_map[dist]=re[0][0]


Matching SOUTHERN-----
Matching SOUTHEAST-----
Matching NORTHERN-----
Matching WESTERN-----
Matching SOUTHWEST-----
Matching NORTHWEST-----
Matching NORTHEAST-----
Matching EASTERN-----
Matching CENTRAL-----
Matching UNKNOWN-----


In [109]:
def find_dist(pp):
    name=None
    for feature in dis_geojson['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(pp):
            return feature['properties']['dist_name']
        else:
            continue
    return name
        

In [111]:
df_nodup['New_District']=df_nodup.District.map(dist_name_map)

In [112]:
df_nodup.loc[df_nodup.District=='UNKNOWN','New_District']=df_nodup[(df_nodup.District=='UNKNOWN')|df_nodup.District.isna()].apply(lambda x: find_dist(Point(x.Longitude, x.Latitude)), axis=1)

In [117]:
df_nodup['New_District'].value_counts()

Northeastern    44482
Southeastern    41913
Central         34186
Southern        33150
Northern        32915
Southwestern    30620
Northwestern    29306
Eastern         25819
Western         23652
Name: New_District, dtype: int64

In [113]:
df_nodup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296586 entries, 0 to 296585
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   CrimeDate         296586 non-null  object 
 1   CrimeTime         296561 non-null  object 
 2   CrimeCode         296586 non-null  object 
 3   Location          295207 non-null  object 
 4   Description       296586 non-null  object 
 5   Inside/Outside    255272 non-null  object 
 6   Weapon            59666 non-null   object 
 7   Post              294212 non-null  object 
 8   District          296586 non-null  object 
 9   Neighborhood      286259 non-null  object 
 10  Longitude         294900 non-null  float64
 11  Latitude          294900 non-null  float64
 12  Premise           260538 non-null  object 
 13  Total Incidents   296586 non-null  int64  
 14  group_count       296586 non-null  int64  
 15  Neighborhood_New  295911 non-null  object 
 16  New_District      29

In [118]:
df_nodup.to_csv('cleaned_df.csv',index=False)